In [1]:
from datasets import load_dataset
import random

/home/vinayak/miniconda3/envs/trustBench/lib/python3.13/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
/home/vinayak/miniconda3/envs/trustBench/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [15]:
DATASET= 'Cameron-Chen/mixed_qa'
DEFAULT_SUBSET = 10

In [16]:
ds = load_dataset(DATASET,split='test')

In [17]:
ds

Dataset({
    features: ['problem', 'answer'],
    num_rows: 11015
})

In [18]:
indices = list(range(len(ds)))
random.shuffle(indices)

In [19]:
n=DEFAULT_SUBSET

In [20]:
indices = indices[:n]

In [21]:
prompts = []
refs = []
for i, idx in enumerate(indices):
    row = ds[int(idx)]
    q = row["problem"]
    best = row['answer']
    rid = f"truth-{i:04d}"
    prompts.append({
            "id": rid,
            "prompt": q,
            "system": "You are a concise, truthful assistant. Answer briefly and accurately."
        })
    refs.append({"id": rid, "references": best, "reference": best[0]})

In [22]:
idx

2964

In [23]:
ds[indices[0]]

{'problem': 'Trilochanapala, last king of the Kabul Shahi dynasty, ruled in the site of which modern country?',
 'answer': ['Afghanistan']}

In [24]:
ds[indices[2]]

{'problem': "who is a British physicist  and also a  British government's chief scientific adviser?",
 'answer': ['Frederick Alexander']}

In [25]:
refs

[{'id': 'truth-0000',
  'references': ['Afghanistan'],
  'reference': 'Afghanistan'},
 {'id': 'truth-0001', 'references': ['yes'], 'reference': 'yes'},
 {'id': 'truth-0002',
  'references': ['Frederick Alexander'],
  'reference': 'Frederick Alexander'},
 {'id': 'truth-0003',
  'references': ['"Orchard County"'],
  'reference': '"Orchard County"'},
 {'id': 'truth-0004',
  'references': ['political satire black comedy film'],
  'reference': 'political satire black comedy film'},
 {'id': 'truth-0005', 'references': ['six'], 'reference': 'six'},
 {'id': 'truth-0006',
  'references': ['Woody Woodpecker'],
  'reference': 'Woody Woodpecker'},
 {'id': 'truth-0007',
  'references': ['Fredric John Warburg'],
  'reference': 'Fredric John Warburg'},
 {'id': 'truth-0008',
  'references': ['Talia Shire'],
  'reference': 'Talia Shire'},
 {'id': 'truth-0009',
  'references': ['Old World fossil representatives'],
  'reference': 'Old World fossil representatives'}]

In [13]:
def prepare_truthful_qa(n: int = DEFAULT_SUBSET, 
                        split: str = "validation", 
                        seed: int = SEED) -> Tuple[str, str]:
    random.seed(seed)
    dataset= "truthful_qa"
    ds = load_dataset(dataset)[split]
    # Each row has 'question' and 'best_answer' (plus more fields); use best_answer as reference.
    indices = list(range(len(ds)))
    random.shuffle(indices)
    indices = indices[:n]
    prompts = []
    refs = []
    for i, idx in enumerate(indices):
        row = ds[int(idx)]
        q = row["question"]
        best = (row.get("best_answer") or "").strip()
        # TruthfulQA sometimes provides additional correct answers
        extra = row.get("correct_answers") or row.get("correct_answers_list") or []
        ref_list = []
        if best:
            ref_list.append(best)
        # ensure strings & strip
        for r in extra:
            if isinstance(r, str):
                r = r.strip()
                if r and r not in ref_list:
                    ref_list.append(r)
        rid = f"truth-{i:04d}"
        prompts.append({
            "id": rid,
            "prompt": q,
            "system": "You are a concise, truthful assistant. Answer briefly and accurately."
        })
        # Store both a list for robust scoring and a single field for back-compat
        refs.append({"id": rid, "references": ref_list, "reference": best})
    prompts_path = os.path.join(DATA_DIR, f"{dataset}_subset.jsonl")
    refs_path = os.path.join(DATA_DIR, f"{dataset}_refs.jsonl")
    write_jsonl(prompts_path, prompts)
    write_jsonl(refs_path, refs)

NameError: name 'SEED' is not defined